In [3]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.linalg import qr as scipy_qr

import cupy as cp

from cupyx.scipy.sparse.linalg import LinearOperator as CuPyLinearOperator
from cupyx.scipy.sparse.linalg import aslinearoperator as cupyaslinearoperator
from scipy.sparse.linalg import LinearOperator, aslinearoperator

from tracelogdetdiag import CUPY_INSTALLED

# Trace functions

## Explicit trace

In [21]:
def explicit_trace_probe(A):
    """Computes the trace of A using an explicit probe. Requires exactly n matvecs with A.
    """

    # Setup
    n = A.shape[0]
    diagonal = np.zeros(n)
    
    # Handle CuPy
    if CUPY_INSTALLED:
        if isinstance(A, CuPyLinearOperator):
            xp = cp
        else:
            xp = np
    else:
        xp = np
    
    for j in range(n):

        # jth column of the identity
        w = xp.zeros(n)
        w[j] = 1.0

        # Compute w^T A w
        diagonal[j] = w.T @ (A @ w)

    trace = xp.sum(diagonal)

    return trace

In [17]:
n = 1000
A = cp.diag(cp.ones(n))
A = cupyaslinearoperator(A)

In [18]:
%%timeit
explicit_trace_probe(A)

150 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
n = 1000
A = np.diag(np.ones(n))
A = aslinearoperator(A)

In [20]:
%%timeit
explicit_trace_probe(A)

1.49 s ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Hutchinson trace

In [40]:
def hutchinson_trace(A, sample_size=100, block_size=20, method="rademacher", exact_sample_size=False):
    """Computes the Hutchinson randomized estimator of tr(A). A must be SPSD.
    
    Here we compute the estimator with sample_size using blocks of samples of size ceil(sample_size/block_size).
    This helps control memory usage vs. vectorization. We don't throw away any samples, so the estimator may be
    computed with a slightly larger sample size than specified, unless exact_sample_size=True.
    """

    # Get shape
    n = A.shape[0]

    valid_methods = ["standard_gaussian", "rademacher"]
    assert method in valid_methods, f"method must be one of {valid_methods}"
    
    # Handle CuPy
    if CUPY_INSTALLED:
        if isinstance(A, CuPyLinearOperator):
            xp = cp
        else:
            xp = np
    else:
        xp = np

    # Handle blocks
    n_blocks = int(xp.ceil(sample_size/block_size))
    extra_samples = (block_size*n_blocks) - sample_size

    block_sums = []
    for j in range(n_blocks):

        # Draw random block of vectors
        if method == "standard_gaussian":
            w = xp.random.normal(size=(n, block_size))
        elif method == "rademacher":
            w = xp.random.choice([-1, 1], size=(n, block_size))
        else:
            raise NotImplementedError
        
        if (j == n_blocks - 1) and (exact_sample_size == True):
            w = w[:,:-extra_samples]
        
        # Append block sum
        block_sum = xp.sum( ( (A.T @ w).T * w.T ).sum(axis=1)  )
        block_sums.append(block_sum)

    tot_sum = xp.sum(xp.asarray(block_sums))
    if exact_sample_size:
        estimate = tot_sum/sample_size
    else:
        estimate = tot_sum/(block_size*n_blocks)

    return estimate

In [41]:
n = 1000
A = cp.diag(cp.ones(n))
A = cupyaslinearoperator(A)

In [43]:
hutchinson_trace(A, sample_size=100)

array(1000.)

In [55]:
n = 1000
A = np.diag(np.ones(n))
A = np.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = aslinearoperator(A)

In [56]:
hutchinson_trace(A, sample_size=100)

996328.7208544658

In [57]:
np.trace(A_explicit)

998738.7970635407

## Hutchinson epsilon delta trace

In [59]:
def hutchinson_epsilon_delta_trace(A, epsilon=0.05, delta=0.05, method="rademacher", block_size=20):
    """Computes an (epsilon, delta)-estimator of trace(A). A must be SPSD. This uses lower-bounds from the literature to pick a sample size 
    for the Hutchinson estimator \hat{tr}(A) such that | \hat{tr}(A) - tr(A) | < epsilon*tr(A) with probability greater than 1 - delta."""
    
    valid_methods = ["standard_gaussian", "rademacher"]
    assert method in valid_methods, f"method must be one of {valid_methods}"

    c = (1.0/(epsilon**2))*np.log(2/delta)

    if method == "standard_gaussian":
        sample_size = int(np.ceil(8*c))
    elif method == "rademacher":
        sample_size = int(np.ceil(6*c))
    else:
        raise NotImplementedError

    return hutchinson_trace(A, sample_size=sample_size, method=method, block_size=block_size)

In [63]:
n = 1000
A = cp.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = cupyaslinearoperator(A)

In [64]:
hutchinson_epsilon_delta_trace(A)

array(1001181.10634622)

In [65]:
cp.trace(A_explicit)

array(1000602.81951611)

In [66]:
n = 1000
A = np.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = aslinearoperator(A)

In [67]:
hutchinson_epsilon_delta_trace(A)

997953.8389890403

In [69]:
np.trace(A_explicit)

998479.5191945141

## Hutchinson plus plus trace

In [82]:
def hutch_plus_plus_trace(A, sample_size=30, method="rademacher"):
    """Computes the Hutch++ randomized estimator of tr(A). A must be SPSD. This is an improved estimator over
    the Hutchinson estimator. See [9].
    
    sample_size must be a multiple of 3.
    """

    # Get shape
    n = A.shape[0]
    
    # Handle CuPy
    if CUPY_INSTALLED:
        if isinstance(A, CuPyLinearOperator):
            xp = cp
        else:
            xp = np
    else:
        xp = np

    valid_methods = ["standard_gaussian", "rademacher"]
    assert method in valid_methods, f"method must be one of {valid_methods}"

    assert sample_size % 3 == 0, "sample_size must be a multiple of 3."
    
    if method == "rademacher":
        S = xp.random.choice([-1, 1], size=(n, int(sample_size/3)))
        G = xp.random.choice([-1, 1], size=(n, int(sample_size/3)))
    elif method == "standard_gaussian":
        S = xp.random.normal(size=(n, int(sample_size/3)))
        G = xp.random.normal(size=(n, int(sample_size/3)))
    else:
        raise NotImplementedError

    # Do QR decomp
    if xp == np: 
        Q, _ = scipy_qr(A @ S, mode="economic")
    else:
        Q, _ = cp.linalg.qr(A @ S, mode="reduced")

    # Compute approximate trace
    term1 = xp.trace(Q.T @ ( A @ Q ) )
    tmp =  A @ ( G - ( Q @ ( Q.T @ G ) ) )
    tmp2 = G.T @ ( tmp - Q @ ( Q.T @ tmp ) )
    term2 = (3/sample_size)*xp.trace(tmp2)
    trace_estimate = term1 + term2
    
    return trace_estimate

In [83]:
n = 1000
A = np.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = aslinearoperator(A)

In [84]:
hutch_plus_plus_trace(A)

995198.0410103138

In [85]:
np.trace(A_explicit)

1000590.7331808059

In [86]:
n = 1000
A = cp.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = cupyaslinearoperator(A)

In [87]:
hutch_plus_plus_trace(A)

array(993471.93523797)

## Hutch plus plus

In [90]:
def hutch_plus_plus_epsilon_delta_trace(A, epsilon=0.05, delta=0.05, method="rademacher"):
    """Computes an (epsilon, delta)-estimator of trace(A) using the Hutch++ algorithm. A must be SPSD. This uses lower-bounds from the literature to pick a sample size 
    for the Hutch++ estimator \hat{tr}(A) such that | \hat{tr}(A) - tr(A) | < epsilon*tr(A) with probability greater than 1 - delta. See [9]."""
    
    valid_methods = ["standard_gaussian", "rademacher"]
    assert method in valid_methods, f"method must be one of {valid_methods}"

    sample_size = int( np.ceil( (np.sqrt(np.log(1/delta))/epsilon) + np.log(1/delta) ) )
    sample_size = int(3*np.ceil(sample_size/3))

    return hutch_plus_plus_trace(A, sample_size=sample_size, method=method)

In [91]:
n = 1000
A = cp.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = cupyaslinearoperator(A)

In [92]:
hutch_plus_plus_epsilon_delta_trace(A)

array(1001249.73319405)

In [94]:
n = 1000
A = np.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = aslinearoperator(A)

In [95]:
hutch_plus_plus_trace(A)

1007033.9071842971

# Logdet

## Explicit -- NOT supported

## Util

## Stochastic Chebyshev

In [97]:
from scipy.sparse.linalg import eigs as scipy_eigs
from cupyx.scipy.sparse.linalg import eigsh as cupy_eigsh

In [104]:

def evaluate_ith_chebyshev_polynomial(xs, i):
    """Evaluates the ith Chebyshev polynomial at imputs xs."""
    tprev = np.ones_like(xs)
    tnext = xs.copy()
    if i == 0:
        return tprev
    elif i == 1:
        return tnext
    else:
        k = 1
        while k < i:
            tnew = 2*xs*tnext - tprev
            tprev = tnext
            tnext = tnew
            k += 1
        return tnew
    


def get_chebyshev_coeff(f, n, i):
    """Computes the ith Chebyshev coefficient in the expansion
            f(x) \approx \sum_{j=0}^n c_j T_j(x).
    Here f:[-1,1] \to \mathbb{R}. 
    """

    # Get nodes
    ks = np.arange(n+1)
    xks = np.cos(  np.pi*( ks + 0.5 )/(n+1)  )

    if i == 0:
        chebyshev_coeff = (1/(n+1))*(f(xks)*1).sum()
    else:
        chebyshev_coeff = (2/(n+1))*(f(xks)*evaluate_ith_chebyshev_polynomial(xks, i) ).sum()
        

    return chebyshev_coeff




In [105]:
def logdet_stochastic_chebyshev_approx(C, sigma_max=None, sigma_min=None, sample_size=100, chebyshev_n=14):
    """Computes an approximation to logdet(C) for a SPSD matrix C, using the 
    stochastic Chebyshev approximation detailed in [7]. Eigenvalues of C are assumed to lie in
    the interval [sigma_min, sigma_max].

    Modified from author code here: https://alinlab.kaist.ac.kr/publications.html.
    """

    # Get dimension
    d = C.shape[0]
    
    # Handle CuPy
    if CUPY_INSTALLED:
        if isinstance(A, CuPyLinearOperator):
            xp = cp
        else:
            xp = np
    else:
        xp = np
    
    if (sigma_max is None) and (sigma_min is None):

        # Get largest and smallest singular values
        if xp == np:
            sigma_max, _ = scipy_eigs(C, k=1, which="LM")
            sigma_min, _ = scipy_eigs(C, k=1, which="SM")
            sigma_max, sigma_min = np.real(sigma_max[0]), np.real(sigma_min[0])
        else:
            sigma_max, _ = cupy_eigsh(C, k=1, which="LM")
            sigma_min, _ = cupy_eigsh(C, k=1, which="SA")
            sigma_max, sigma_min = cp.real(sigma_max[0]), cp.real(sigma_min[0])
            sigma_max, sigma_min = cp.asnumpy(sigma_max), cp.asnumpy(sigma_min) 

    # Scaling
    a = sigma_min + sigma_max
    delta = sigma_min/a

    # Make B
    B = (1/a)*C
    logdet_estimate = 0.0

    # Funcs
    f = lambda x: np.log(1-x)
    g = lambda x: ((1-2*delta)/2)*x + 0.5
    ginv = lambda x: (2/(1-2*delta))*x
    h = lambda x: f(g(x))

    # Get Chebyshev coeffs
    chebyshev_coeffs = [ get_chebyshev_coeff(h, chebyshev_n, i) for i in range(0, chebyshev_n+1) ]

    # Random sampling
    for j in range(sample_size):
        
        # Draw random vector
        v = xp.random.choice([-1, 1], size=d)
        u = chebyshev_coeffs[0]*v

        if chebyshev_n > 1:
            w0 = v
            w1 = B @ v
            w1 = ginv(w1)
            w1 = v/(1 - 2*delta) - w1
            u = chebyshev_coeffs[1]*w1 + chebyshev_coeffs[0]*w0

            for k in range(2, chebyshev_n+1):

                w2 = B @ w1
                w2 = ginv(w2)
                w2 = w1/(1 - 2*delta) - w2
                w2 = 2*w2 - w0
                u = chebyshev_coeffs[k]*w2 + u
                w0 = w1
                w1 = w2
        
        logdet_estimate += (xp.dot(v, u))/sample_size

    logdet_estimate += d*xp.log(a)

    return logdet_estimate



# def logdet_stochastic_chebyshev_epsilon_delta_approx(C, epsilon=0.1, zeta=0.1, sample_size=None, details=False):
#     """Computes an approximation to logdet(C) for a SPD matrix C, using the 
#     stochastic Chebyshev approximation detailed in [7]. Returns an estimate
#     \hat{logdet}(C) s.t. |logdet(C) - \hat{logdet}(C)| < epsilon*|logdet(C)| 
#     with at least probaility 1-zeta. 

#     If you override sample_size (which you might do since the bound is loose), you
#     no longer have the same guarantee.

#     Modified from author code here: https://alinlab.kaist.ac.kr/publications.html.
#     """

#     # Get largest and smallest singular values
#     sigma_max, _ = scipy_eigs(C, k=1, which="LM")
#     sigma_min, _ = scipy_eigs(C, k=1, which="SM")
#     sigma_max, sigma_min = np.real(sigma_max[0]), np.real(sigma_min[0])
#     kappa = sigma_max/sigma_min

#     # Compute M and N
#     M = (14/(epsilon**2))*((np.log(1 + (kappa**2)))**2)*np.log(2/zeta) # lower bound on sample size
#     N_denom = np.log( ( np.sqrt(2*(kappa**2) + 1) + 1  ) / ( np.sqrt(2*(kappa**2) + 1) - 1 )  )
#     N_num = np.log( (20/epsilon)*( np.sqrt( 2*(kappa**2) + 1 ) - 1 )*( (np.log(2 + 2*(kappa**2))) / (np.log(1 + (1/(kappa**2))))  )  )
#     N = N_num/N_denom # chebyshev_n

#     M = int(np.ceil(M))
#     N = int(np.ceil(N))

#     if details == True:
#         print(f"Using {M} samples.")
#         print(f"Using Chebyshev polynomials of order {N}.")

#     return logdet_stochastic_chebyshev_approx(C, sigma_max, sigma_min, sample_size=M, chebyshev_n=N)




In [106]:
n = 1000
A = cp.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = cupyaslinearoperator(A)

In [113]:
logdet_stochastic_chebyshev_approx(A, sample_size=1000, chebyshev_n=30)

array(5950.97281134)

In [110]:
def logdet_via_cholesky(A, banded_cholesky=False):
    """Computes logdet(A) using the Cholesky method. A must be SPD."""

    if not banded_cholesky:
        chol = np.linalg.cholesky(A)
    else:
        raise NotImplementedError

    return 2*np.sum(np.log(np.diag(chol)))

In [111]:
logdet_via_cholesky(cp.asnumpy(A_explicit))

5903.710338458119

In [114]:
n = 1000
A = np.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = aslinearoperator(A)

In [115]:
logdet_stochastic_chebyshev_approx(A, sample_size=1000, chebyshev_n=30)

5950.228124871075

# Diag

## Explicit

In [117]:
def explicit_diag_probe(A):
    """Computes the diagonal of A using an explicit probe. Requires exactly n matvecs with A.
    """

    # Handle CuPy
    if CUPY_INSTALLED:
        if isinstance(A, CuPyLinearOperator):
            xp = cp
        else:
            xp = np
    else:
        xp = np
    
    # Setup
    n = A.shape[0]
    diagonal = xp.zeros(n)

    for j in range(n):

        # jth column of the identity
        w = xp.zeros(n)
        w[j] = 1.0

        # Compute w^T A w
        diagonal[j] = w.T @ (A @ w)

    return diagonal

In [118]:
n = 1000
A = cp.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = cupyaslinearoperator(A)

In [120]:
np.linalg.norm(explicit_diag_probe(A) - cp.diag(A_explicit))

array(0.)

## Naive diag

In [121]:
def naive_diag(A, sample_size=1000):
    """Naive unbiased estimator for the diagonal of a matrix, see [5]. A must be SPSD.
    """

    # Get shape
    n = A.shape[0]
    
    # Handle CuPy
    if CUPY_INSTALLED:
        if isinstance(A, CuPyLinearOperator):
            xp = cp
        else:
            xp = np
    else:
        xp = np

    diag_estimate = xp.zeros(n)
    tk = xp.zeros(n)
    qk = xp.zeros(n)

    for j in range(sample_size):
        
        # Draw random vector
        vk = xp.random.choice([-1, 1], size=n)

        # Update tk
        tk = tk + ((A @ vk) * vk)

        # Update qk
        qk = qk + (vk*vk)

        # Update diag_estimate
        diag_estimate = tk / qk

    return diag_estimate

In [122]:
n = 1000
A = cp.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = cupyaslinearoperator(A)

In [127]:
np.linalg.norm(naive_diag(A,  sample_size=10000) - cp.diag(A_explicit))/n

array(0.31204026)

# Diaginv

## Explicit

In [ ]:
def explicit_diaginv_probe(A, method="cholesky"):
    """Computes the diagonal of inv(A) using an explicit probe. A must be SPD.
    """

    valid_methods = ["cholesky"]
    assert method in valid_methods, f"method must be one of {valid_methods}"

    # Setup
    n = A.shape[0]
    diagonal_inv = np.zeros(n)

    if method == "cholesky":
        chol = scipy_chol_fac(A)
    else:
        raise NotImplementedError

    for j in range(n):

        # jth column of the identity
        w = np.zeros(n)
        w[j] = 1.0

        # Compute w^T inv(A) w
        if method == "cholesky":
            Ainv_w = scipy_chol_solve(chol, w)
        else:
            raise NotImplementedError

        diagonal_inv[j] = w.T @ Ainv_w

    return diagonal_inv

# Trace of inverse?

In [5]:
from tracelogdetdiag.util import AinvCGLinearOperator
from tracelogdetdiag.util import AinvCGCuPyLinearOperator

In [6]:
n = 1000
A = cp.random.normal(size=(n,n))
A = A.T @ A
A_explicit = A
A = cupyaslinearoperator(A)

In [10]:
Ainv = AinvCGCuPyLinearOperator(A, cg_tol=1e-2, cg_maxits=2000)

In [11]:
z = cp.ones(n)

In [13]:
Ainv_sol = Ainv @ z

In [14]:
Ainv_sol = cp.asnumpy(Ainv_sol)

In [15]:
A_explicit = cp.asnumpy(A_explicit)

In [16]:
np_sol = np.linalg.solve(A_explicit, cp.asnumpy(z))

In [17]:
np.linalg.norm(Ainv_sol - np_sol)

0.0004166200113511358